<a href="https://colab.research.google.com/github/MudasirIrshad/Agentic_AI_Panaversity_Course/blob/main/Function_Calling_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install -qU "google-generativeai"

In [ ]:
from google.colab import userdata
import google.generativeai as genai

import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
genai.configure()

In [ ]:
def enable_lights():
  """Turn on the lighting system"""
  print("LIGHTBOT: Lights enabled.")

def set_light_color(rgb_hex: str):
  """Set the light color, lights must be enabled for this to work."""
  print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def disable_lights():
  """Turn off the lighting system"""
  print("LIGHTBOT: Lights disabled.")

In [ ]:
light_controls = [enable_lights, set_light_color,disable_lights]
instruction = """you are a helpful lightning system bot. You can turn lights on
and off and you can set color of lights. Do not perform any other task"""

model = genai.GenerativeModel(
    "gemini-2.5-flash", tools= light_controls, system_instruction=instruction
)

In [ ]:
# chat = model.start_chat()

# res = chat.send_message("set light color to red")
# res.text

#Practicing using Dictionary

In [ ]:
students_details={
    "111": {
        "name": "Mudasir Irshad",
        "age": 20,
        "grade": "A"
    },
    "112": {
        "name": "Haris",
        "age": 22,
        "grade": "B"
    },
    "113": {
        "name": "Shoaib",
        "age": 21,
        "grade": "C"
    },
    "114": {
        "name": "Ajlal",
        "age": 23,
        "grade": "A"
    }
}

students_marks={
    "111":{
        "english": 50,
        "math": 60,
        "science": 70
    },
    "112":{
        "english": 60,
        "math": 70,
        "science": 80
    },
    "113":{
        "english": 70,
        "math": 80,
        "science": 90
    },
    "114":{
        "english": 80,
        "math": 90,
        "science": 100
    }
}

print(students_marks.get("112"))

In [ ]:
def get_student_details(student_id:str):
  """Get student detail from students_details dictionary"""
  print("SCHOOLBOT: Student details retrieved.")
  return students_details.get(student_id)

def get_student_marks(student_id: str):
  """Get student marks from students_marks dictionary"""
  print("SCHOOLBOT: Student marks retrieved.")
  return students_marks.get(student_id)

def get_student_marks_and_details(student_id: str):
  """Get student marks and details from students_details and students_marks dictionary"""
  print("SCHOOLBOT: Student marks and details retrieved.")
  return students_details.get(student_id), students_marks.get(student_id)

instruction = """
You are a helpful school's student's details retrieval bot.
You can get specific student details by checking student id.
you can get specific student marks by checking student id.
you can get specific student marks and details by checking student id.
Do not perform any other tasks.
"""


model = genai.GenerativeModel(
    "gemini-2.0-flash-001", tools=[get_student_details,get_student_marks,get_student_marks_and_details], system_instruction=instruction
)



In [ ]:
# chat = model.start_chat()

# response = chat.send_message("get student details and marks of student id 111")

# function_name = response.candidates[0].content.parts[0].function_call.name
# student_id = response.candidates[0].content.parts[0].function_call.args["student_id"]

# if function_name == "get_student_details":
#   print(get_student_details(student_id))
# elif function_name == "get_student_marks":
#   print(get_student_marks(student_id))
# elif function_name == "get_student_marks_and_details":
#   print(get_student_marks_and_details(student_id))

#Exploring MODES in function_calling


1.   NONE
2.   ANY
3.   AUTO



In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
  """Create a tool config with the specified function calling mode."""
  return content_types.to_tool_config(
      {"function_calling_config":{"mode":mode, "allowed_function_names":fns}}
  )

#Automate the process of function calling with result instead of using manuall if else to get specific result from any function
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

tool_config = tool_config_from_mode("any")
res = chat.send_message("First give me detail of student then his marks then both of student id 114", tool_config=tool_config)


# 3 things we need in creating a function


These three things would be in a single DOC String """xxxx"""
*   Description: "What need to be done by AI"
*   Arguments (Arg): "what are required to call a function"
*   Return: "Output generated by function"



In [ ]:
def students_remarks_by_teacher(student_id: str):

  """
  get student name and marks for remark generation
  """

  details = students_details.get(student_id)
  marks = students_marks.get(student_id)

  return {
      "name": details["name"],
      "marks": marks
  }

In [ ]:
remarkBootinstruction = """you are a helpful Student Remark's Boot.
when asked for student's remarks:
1. call students_remarks_by_teacher to get name and marks.
2. Analyze the marks and write short remarks about it.
3. Remarks must be constructive even if marks are bad.
4. Return a good supportive string with remarks
Do not perform any other task"""

new_model =  genai.GenerativeModel(
    "gemini-2.5-flash", tools=[students_remarks_by_teacher], system_instruction=remarkBootinstruction
)

In [ ]:
remark_boot_chat = new_model.start_chat(enable_automatic_function_calling=True)
remark_boot_response = remark_boot_chat.send_message('remarks of student id "111"')
remark_boot_response.text